In [1]:
#spark avec python
!pip install pyspark
!pip install findspark

In [2]:
### config
from pyspark.sql import SparkSession # required to created a dataframe
spark=SparkSession.builder.appName("Basics").getOrCreate() 
#spark=SparkSession.builder.appName("Basics").config("spark.executor.memory", "70g").config("spark.driver.memory", "50g").config("spark.memory.offHeap.enabled",True).config("spark.memory.offHeap.size","16g").getOrCreate() 

import pyspark.sql.functions

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/22 15:35:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/opt/conda/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [3]:
file = '../input/bitcoin-tweets-20160101-to-20190329/tweets.csv'
df = spark.read.csv(file, sep=";",inferSchema=True, header=True,multiLine=True)

In [4]:
n_rows=df.count()
n_cols = len(df.columns)


In [5]:
#colonnes 
print(n_rows,n_cols)
df.printSchema()

17026912 9
root
 |-- id: string (nullable = true)
 |-- user: string (nullable = true)
 |-- fullname: string (nullable = true)
 |-- url: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- replies: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- retweets: string (nullable = true)
 |-- text: string (nullable = true)



In [6]:
# df=df.drop("Label")

In [7]:
df.head(10)

[Row(id='1132977055300300800', user='KamdemAbdiel', fullname='Abdiel kamdem', url=None, timestamp='2019-05-27 11:49:14+00', replies='0', likes='0', retweets='0', text='È appena uscito un nuovo video! LES CRYPTOMONNAIES QUI PULVÉRISENT BITCOIN EN 2019 https://t.co/yCsQMvRnyS'),
 Row(id='1132977073402736640', user='bitcointe', fullname='Bitcointe', url=None, timestamp='2019-05-27 11:49:18+00', replies='0', likes='0', retweets='0', text='Cardano: Digitize Currencies; EOS https://t.co/1kTKqKEBlS 6500% ROI; AT&amp;T Bitcoin Bill Pay https://t.co/eQCwOXKHK0   |  Cardano (ADA) 🌏📢😎🤑💵 | #FolloForFolloBack #follo4folloback #followforfollow #bitcointe #cryptocurrency'),
 Row(id='1132977023893139456', user='3eyedbran', fullname='Bran - 3 Eyed Raven', url=None, timestamp='2019-05-27 11:49:06+00', replies='0', likes='2', retweets='1', text="Another Test tweet that wasn't caught in the stream ! bitcoin"),
 Row(id='1132977089089556481', user='DetroitCrypto', fullname='J. Scardina', url=None, timestamp

In [8]:
from pyspark.sql.functions import col
# 
df = df.withColumn("likes", col("likes").cast("integer"))
df = df.withColumn("retweets", col("retweets").cast("integer"))
df = df.withColumn("replies", col("retweets").cast("integer"))

In [9]:
#suprimmer tweet vides (retweets sans caption)
df=df.filter(~df["text"].isNull())

In [10]:
like_sort=df.orderBy(df["likes"].desc(),df["retweets"].desc())

In [11]:
top_likes=like_sort.select("timestamp","likes","retweets","text").head(10)

In [12]:
top_likes

[Row(timestamp='2019-04-04 02:07:05+00', likes=291985, retweets=61585, text='The type of accidents I’m tryna make https://t.co/Z47ImFA14m'),
 Row(timestamp='2018-01-24 17:29:19+00', likes=282959, retweets=64013, text="FRIENDS 2018 PLOTLINES\n-Joey eats Tide pods, goes to hospital (guest star Ellen Pompeo)\n-Chandler won't shut up about how funny his Vines were\n-Ed Sheeran cameo interrupts Phoebe's Central Perk set \n-Ross gets everyone into Bitcoin\n-The One Where The Gang Realizes Their Privilege"),
 Row(timestamp='2018-01-16 23:02:59+00', likes=138468, retweets=34318, text='if you lost money on bitcoin today target is giving away $5 giftcards to anyone that gets a flu shot. on my 8th shot today. feel great. fuck a 9-5'),
 Row(timestamp='2017-12-09 03:47:00+00', likes=130455, retweets=22002, text='Wow, if I had invested $1,000 in Bitcoin last week, today I would have... still no idea how Bitcoin works.'),
 Row(timestamp='2017-12-23 23:30:42+00', likes=124513, retweets=26413, text='th

In [13]:
top_likes[0].text

'The type of accidents I’m tryna make https://t.co/Z47ImFA14m'

In [14]:
#code to remove url
from urllib.parse import urlparse
import re

def remove_urls(s):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    # Replace URLs with an empty string
    return url_pattern.sub('[url]', s)


In [15]:
pip install googletrans==4.0.0-rc1

Note: you may need to restart the kernel to use updated packages.


In [16]:
#code to detect and translate language 
from googletrans import Translator
from nltk.tokenize import sent_tokenize
def translate_to_english(text):
    # Create a Translator object
    translator = Translator()

    # Detect the language of the text
    detected_lang = translator.detect(text).lang

    # If the detected language is not English, translate the text to English
    if detected_lang != 'en':
#         print("detected language",detected_lang) #out language 
        translated_text = translator.translate(text, src=detected_lang, dest='en').text
    else:
        translated_text = text

    return translated_text

# Example usage
def preprocess(text):
    sentences=sent_tokenize(text)
    out= ' '.join([translate_to_english(remove_urls(sent)) for sent in sentences])
    return out

In [17]:
common_time=df.groupBy("timestamp").count()

In [18]:
common_time.orderBy(col("count").desc()).where(col('timestamp').isNotNull()).show()

24/05/22 15:39:45 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/22 15:40:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+--------------------+-----+
|           timestamp|count|
+--------------------+-----+
|2019-06-04 17:41:...|  102|
|2019-06-16 08:47:...|   99|
|2019-06-29 02:01:...|   85|
|2019-06-21 08:01:...|   79|
|2019-07-19 13:05:...|   73|
|2019-07-20 16:58:...|   71|
|2019-06-04 00:02:...|   67|
|2019-06-04 22:24:...|   67|
|2019-06-03 14:49:...|   66|
|2019-07-15 18:20:...|   66|
|2019-07-09 16:05:...|   63|
|2019-07-13 18:18:...|   63|
|2019-07-05 19:50:...|   62|
|2019-07-08 09:00:...|   61|
|2019-06-28 07:01:...|   61|
|2019-06-04 22:00:...|   60|
|2019-07-02 08:48:...|   60|
|2019-06-02 13:18:...|   59|
|2019-06-29 16:12:...|   59|
|2019-07-10 01:30:...|   58|
+--------------------+-----+
only showing top 20 rows



In [19]:
popular=df.filter(col('timestamp').contains("2019-06-04 17:41:"))

In [20]:
popular.select("id","timestamp","text").head(10)
#bots et spam 

[Row(id='1135964769737986048', timestamp='2019-06-04 17:41:20+00', text='#btc #btc rumor winkles  want out \n\nthought they were already out \n\n#crypto😍urrencies $gbtc 😍😍😍😍😍😍'),
 Row(id='1135964688536260613', timestamp='2019-06-04 17:41:01+00', text='⏰ LIQUIDATION on BTC-27SEP19 ☠️️\n\n Sold $200 of #BTC @ $7764.00\n 4 Jun 2019 17:40:11 UTC\nTrade ID: 23243314'),
 Row(id='1135964688515325956', timestamp='2019-06-04 17:41:01+00', text='⏰ LIQUIDATION on BTC-27SEP19 ☠️️\n\n Bought $5,060 of #BTC @ $7763.25\n 4 Jun 2019 17:40:11 UTC\nTrade ID: 23243330\n\nThose who seek the Lord lack no good thing.'),
 Row(id='1135964688594952194', timestamp='2019-06-04 17:41:01+00', text='⏰ LIQUIDATION on BTC-27SEP19 ☠️️\n\n Sold $5,000 of #BTC @ $7736.50\n 4 Jun 2019 17:40:19 UTC\nTrade ID: 23243786\n\nSome people like taking the long way home.'),
 Row(id='1135964688678871041', timestamp='2019-06-04 17:41:01+00', text='⏰ LIQUIDATION on BTC-27SEP19 ☠️️\n\n Bought $330 of #BTC @ $7763.25\n 4 Jun 2019 17:4

In [21]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline
# import os
# finbert = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
# tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
# classifier = pipeline('sentiment-analysis', model=finbert, tokenizer=tokenizer)

In [25]:
# nltk.download("vader_lexicon")
from nltk.sentiment import SentimentIntensityAnalyzer

def sentiment_analysis(text):
    try:
        tweet=preprocess(text)
        sia = SentimentIntensityAnalyzer()  #Création d'une instance de SentimentIntensityAnalyzer() :
        sent = sia.polarity_scores(tweet)  #Calcul du score de sentiment à l'aide de polarity_scores() et récupération du score "compound" :
        return(tweet,sent)

    except Exception as e:
        print(e)
        print('text :',text)
        return(text,{'compound': -2.0})
def Label(text):
    score=sentiment_analysis(text)[1]["compound"]
    if score >0.1:
        return "Positive"
    elif score > -0.1:
        return  "Neutral"
    else:
        return "Negative"

for row in df.head(10):
    print(sentiment_analysis(row.text))

/opt/conda/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


('A new video has just come out! The cryptocurrencies that spray Bitcoin in 2019 [URL]', {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0})
('Cardano: Digitize Currencies; EOS [url] 6500% ROI; AT&amp;T Bitcoin Bill Pay [url]   |  Cardano (ADA) 🌏📢😎🤑💵 | #FolloForFolloBack #follo4folloback #followforfollow #bitcointe #cryptocurrency', {'neg': 0.069, 'neu': 0.931, 'pos': 0.0, 'compound': -0.1027})
("Another Test tweet that wasn't caught in the stream ! bitcoin", {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0})
('Current Crypto Prices! BTC: $8721.99 USD\nETH: $266.62 USD\nLTC: $114.03 USD\nBCH: $432.02 USD\nXLM: $0.133 USD\nDOGE: $ 0.00314 USD\nNEO: $12.04 USD\nXRP: $0.4092 USD\nCANN: $0.001482 USD\nEMC2: $0.1198 USD\nXMR: $94.65 USD\nBTG: $24.31 USD', {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0})
('Spiv (Nosar Baz): BITCOIN Is An Asset &amp; NOT A Currency. [url]\n\n[url]\n\n[url]\n\n[url]\n\n[url]', {'neg': 0.0, 'neu': 0.848, 'pos': 0.152, 'compound': 0.3612})
('#BTC br

In [27]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

Sentiment_Label = udf(Label, StringType())

In [28]:
df = df.withColumn("Label", Sentiment_Label(df["text"]))
df.head(10)

# creation de colonne label (sentiment)

/opt/conda/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


[Row(id='1132977055300300800', user='KamdemAbdiel', fullname='Abdiel kamdem', url=None, timestamp='2019-05-27 11:49:14+00', replies=0, likes=0, retweets=0, text='È appena uscito un nuovo video! LES CRYPTOMONNAIES QUI PULVÉRISENT BITCOIN EN 2019 https://t.co/yCsQMvRnyS', Label='Neutral'),
 Row(id='1132977073402736640', user='bitcointe', fullname='Bitcointe', url=None, timestamp='2019-05-27 11:49:18+00', replies=0, likes=0, retweets=0, text='Cardano: Digitize Currencies; EOS https://t.co/1kTKqKEBlS 6500% ROI; AT&amp;T Bitcoin Bill Pay https://t.co/eQCwOXKHK0   |  Cardano (ADA) 🌏📢😎🤑💵 | #FolloForFolloBack #follo4folloback #followforfollow #bitcointe #cryptocurrency', Label='Negative'),
 Row(id='1132977023893139456', user='3eyedbran', fullname='Bran - 3 Eyed Raven', url=None, timestamp='2019-05-27 11:49:06+00', replies=1, likes=2, retweets=1, text="Another Test tweet that wasn't caught in the stream ! bitcoin", Label='Neutral'),
 Row(id='1132977089089556481', user='DetroitCrypto', fullname=

In [ ]:
from pyspark.sql.functions import to_timestamp, hour, sum as sum_

df = df.withColumn("timestamp", to_timestamp(col("timestamp")))

df = df.withColumn("hour", hour(col("timestamp")))

df.show(truncate=False)

/opt/conda/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "
invalid source language
text : @AliveCasino ,@ARFREYR, @ArthurB ,@ZiaNabavi1 ,@AmeerRosic ,@coin_wonder ,@wizkidayo ,@yegg ,@cni ,@bijancbayne ,@bah
'NoneType' object is not iterable
text : 3-5 Kuruş para bulup BİT i kanlanınca bu milleti ,kendisine ağam-paşam dedirtenler kalitesiz ucuz tiplerdir.

#DOLAR
#DolarTL
#bist
#bist100 
#usdtry
#USDTRY
#XU100 
#𝒷𝑒𝓁𝑒𝓃𝓈𝒶𝓎 2012
#doge #dogeusd
#btc #btcusd
YTD
invalid source language                                             (0 + 1) / 1]
text : #Bitcoin #Satoshi #crypto #blockchain #Airdrop
New Airdrop #Inbot 📢

Join the Inbot Ambassador community and earn dividends from AI ✈

(💵 Earn Up $350 per referral)

1. Sign Up Here : https://t.co/bALgmQlZun 🚀

2. +150 #InTokens by referral https://t.co/